In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
import os
import cv2

# Define the path to the main folder
path_to_folder = "Dataset"

# Loop through all subdirectories inside the main folder
for subdir in os.listdir(path_to_folder):

    # Define the path to the subdirectory
    subdir_path = os.path.join(path_to_folder, subdir)

    # Check if the subdirectory is a directory
    if os.path.isdir(subdir_path):

        # Loop through all image files inside the subdirectory
        for file in os.listdir(subdir_path):

            # Define the path to the image file
            file_path = os.path.join(subdir_path, file)

            # Check if the file is an image file
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):

                # Read the image file
                img = cv2.imread(file_path)

                # Apply background removal using the GrabCut algorithm
                # ...
                mask = np.zeros(img.shape[:2], np.uint8)

                # Define the rectangle containing the object you want to keep
                rect = (50,50,450,290)

                # Initialize the background and foreground models using the rectangle
                bgdModel = np.zeros((1, 65), np.float64)
                fgdModel = np.zeros((1, 65), np.float64)
            
                # Apply the GrabCut algorithm to the image to obtain the new mask
                cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

                # Create a new mask with only the foreground pixels set to 1
                new_mask = np.where((mask == 1) + (mask == 3), 255, 0).astype('uint8')

                # Apply the new mask to the original image
                result = cv2.bitwise_and(img, img, mask=new_mask)

                # Save the result
                cv2.imwrite(file_path, result)


In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,validation_split=0.2)

In [55]:
train = train_datagen.flow_from_directory('Dataset',target_size = (224,224), batch_size = 32,class_mode = 'categorical',subset = 'training')

Found 85902 images belonging to 158 classes.


In [56]:
test = train_datagen.flow_from_directory('Dataset',target_size = (224,224), batch_size = 32, class_mode = 'categorical',subset = 'validation')

Found 21405 images belonging to 158 classes.


In [57]:
train.image_shape

(224, 224, 3)

In [58]:
class_names = []
import os

# Define the directory path
directory_path = 'Dataset'

# Get a list of all folders in the directory
folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

# Print the folder names
for folder in folders:
    class_names.append(folder)

In [59]:
train.classes

array([  0,   0,   0, ..., 157, 157, 157])

In [60]:
num = []

In [61]:
for i in range(0,158):
    num.append(i)

In [62]:
for i in range(len(class_names)):
    if class_names[i].startswith("n"):
        class_names[i] = class_names[i][10:]

In [63]:
for i in range(0,60):
    class_names[i] = class_names[i][4:]

In [64]:
len(class_names)

158

In [65]:
class_names

['Black_footed_Albatross',
 'Laysan_Albatross',
 'Sooty_Albatross',
 'Groove_billed_Ani',
 'Crested_Auklet',
 'Least_Auklet',
 'Parakeet_Auklet',
 'Rhinoceros_Auklet',
 'Brewer_Blackbird',
 'Red_winged_Blackbird',
 'Rusty_Blackbird',
 'Yellow_headed_Blackbird',
 'Bobolink',
 'Indigo_Bunting',
 'Lazuli_Bunting',
 'Painted_Bunting',
 'Cardinal',
 'Spotted_Catbird',
 'Gray_Catbird',
 'Yellow_breasted_Chat',
 'Eastern_Towhee',
 'Chuck_will_Widow',
 'Brandt_Cormorant',
 'Red_faced_Cormorant',
 'Pelagic_Cormorant',
 'Bronzed_Cowbird',
 'Shiny_Cowbird',
 'Brown_Creeper',
 'American_Crow',
 'Fish_Crow',
 'Black_billed_Cuckoo',
 'Mangrove_Cuckoo',
 'Yellow_billed_Cuckoo',
 'Gray_crowned_Rosy_Finch',
 'Purple_Finch',
 'Northern_Flicker',
 'Acadian_Flycatcher',
 'Great_Crested_Flycatcher',
 'Least_Flycatcher',
 'Olive_sided_Flycatcher',
 'Scissor_tailed_Flycatcher',
 'Vermilion_Flycatcher',
 'Yellow_bellied_Flycatcher',
 'Frigatebird',
 'Northern_Fulmar',
 'Gadwall',
 'American_Goldfinch',
 'Euro

In [66]:
animal_map = dict(zip(num,class_names))

In [67]:
animal_map[1]

'Laysan_Albatross'

In [143]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set the input shape and number of classes
img_height, img_width = 224, 224
num_classes = 158

# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
model.fit(train,steps_per_epoch = 3,epochs= 9)



Epoch 1/9
3/3 [==============================] - 3s 681ms/step - loss: 4.6976 - accuracy: 0.0208
Epoch 2/9
3/3 [==============================] - 2s 672ms/step - loss: 4.3492 - accuracy: 0.2500
Epoch 3/9
3/3 [==============================] - 2s 672ms/step - loss: 3.9858 - accuracy: 0.5625
Epoch 4/9
3/3 [==============================] - 2s 667ms/step - loss: 4.0741 - accuracy: 0.4688
Epoch 5/9
3/3 [==============================] - 2s 673ms/step - loss: 4.1703 - accuracy: 0.5104
Epoch 6/9
3/3 [==============================] - 2s 684ms/step - loss: 3.9826 - accuracy: 0.4583
Epoch 7/9
3/3 [==============================] - 2s 675ms/step - loss: 3.7348 - accuracy: 0.5104
Epoch 8/9
3/3 [==============================] - 2s 694ms/step - loss: 3.8618 - accuracy: 0.4167
Epoch 9/9
3/3 [==============================] - 2s 670ms/step - loss: 3.0226 - accuracy: 0.6250


In [12]:
from sklearn import metrics as mc

In [144]:
result = model.predict(test)
r = np.argmax(result, axis=-1)
mc.accuracy_score(test.classes,r)

669/669 [==============================] - 223s 334ms/step


0.4954449894884373

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,validation_split=0.2)

In [3]:
train = train_datagen.flow_from_directory('test',target_size = (112,112), batch_size = 32,class_mode = 'categorical',subset = 'training')

Found 467 images belonging to 10 classes.


In [4]:
test = train_datagen.flow_from_directory('test',target_size = (112,112), batch_size = 32, class_mode = 'categorical',subset = 'validation')

Found 115 images belonging to 10 classes.


In [134]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
# Set the input shape and number of classes
img_height, img_width = 112, 112
num_classes = 10

# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
model.fit(train,steps_per_epoch = 467//32,epochs= 50,batch_size = 32,validation_data = test)



Epoch 1/50
14/14 [==============================] - 3s 190ms/step - loss: 2.3106 - accuracy: 0.0851 - val_loss: 2.2987 - val_accuracy: 0.1043
Epoch 2/50
14/14 [==============================] - 2s 175ms/step - loss: 2.3026 - accuracy: 0.1034 - val_loss: 2.2968 - val_accuracy: 0.1043
Epoch 3/50
14/14 [==============================] - 2s 172ms/step - loss: 2.2954 - accuracy: 0.1287 - val_loss: 2.2817 - val_accuracy: 0.1043
Epoch 4/50
14/14 [==============================] - 2s 173ms/step - loss: 2.2960 - accuracy: 0.1264 - val_loss: 2.2845 - val_accuracy: 0.2435
Epoch 5/50
14/14 [==============================] - 2s 173ms/step - loss: 2.2608 - accuracy: 0.1839 - val_loss: 2.1544 - val_accuracy: 0.2609
Epoch 6/50
14/14 [==============================] - 2s 173ms/step - loss: 2.1603 - accuracy: 0.1862 - val_loss: 2.1608 - val_accuracy: 0.1826
Epoch 7/50
14/14 [==============================] - 2s 177ms/step - loss: 2.1437 - accuracy: 0.2299 - val_loss: 2.0639 - val_accuracy: 0.2174
Epoch 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
# Set the input shape and number of classes
img_height, img_width = 112, 112
num_classes = 10

# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 55, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 53, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 26, 26, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0